In [171]:
import pandas as pd
import numpy as np 
import ydata_profiling
import matplotlib.pyplot as plt

# Scalers 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

# Imputers
from sklearn.impute import KNNImputer



In [172]:
data = pd.read_csv('../Datasets/XYZ_sports_dataset.csv', sep=';')

### __First analysis and selection with pandas profiling__

##### 0. Profiling

In [173]:
# Lets make a profiling report
#profile = pandas_profiling.ProfileReport(data)
#profile.to_file(outputfile="XYZ_sports_dataset.html")

##### 1. Constant features
- DanceActivities
- NatureActivities

In [174]:
data = data.drop(['DanceActivities', 'NatureActivities'], axis=1)

##### 2. Quasi-constant features (highly inbalanced)
- OtherActivities (98.0%)
- AtheleticsActivities (93.7%)
- NumberOfReferences (92.4%)

Using a threshold of 90%. Variables not listed have a lower percentage of imbalance.

In [175]:
droped = data[['OtherActivities', 'NumberOfReferences', 'AthleticsActivities']]
data = data.drop(['OtherActivities', 'NumberOfReferences', 'AthleticsActivities'], axis=1)

##### 3. Uniform features that provide no information 
Since all values are unique and they dont provide any kind of ordinal information, they are removed.
- ID

In [176]:
data = data.drop(['ID'], axis=1)

### __Feature name change for better understanding__
Like this, I will be able to access them more easily and understand what they are.
1. Binary will start with a b
2. Numerical will start with n
3. Date will start with d

In [177]:
data.columns = ['n_age', 'b_gender', 'n_income', 'd_enrollmentStart', 'd_enrollmentFinish', 'd_lastPeriodStart', 'd_lastPeriodFinish', 'd_lastVisit', 'n_daysWithoutFrequency', 'n_lifetimeValue',
                'b_useByTime', 'b_waterActivities', 'b_fitnessActivities', 'b_teamActivities', 'b_racketActivities', 'b_combatActivities', 'b_specialActivities', 'n_frequencies',
                'n_attendedClasses', 'n_allowedWeeklyVisits', 'n_allowedVisits', 'n_realVisits', 'n_renewals', 'b_hasReferences', 'b_dropout']

data['b_gender'] = data['b_gender'].map({'Male' : 0, 'Female' : 1})

Lets recheck the imbalances in the binary and numerical features <br><br>
__1. Binary features__

In [178]:
binary_features = [col for col in data.columns if col.startswith('b_')]

balance = {}
for feature in binary_features:
    value_counts = data.value_counts(feature)
    balance[feature] = 100* round(value_counts.min() / value_counts.max(),3)

balance = pd.DataFrame.from_dict(balance, orient='index', columns=['balance'])
balance = balance.sort_values(by='balance', ascending=False)
balance

,balance
b_fitnessActivities,73.6
b_gender,67.3
b_waterActivities,42.1
b_dropout,24.8
b_combatActivities,12.1
b_teamActivities,5.9
b_useByTime,4.9
b_specialActivities,2.7
b_racketActivities,2.4
b_hasReferences,2.0


Segmenting customers in such smalls groups is not the best idea (2% of the data). Lets set the threshold to 3% and remove the features that have less than that. 
We will, of course, keep a backup of the dropped features, because they might be useful for feature engineering:
- Joining the very unlikely activities into one feature might prove useful, but that's an idea for later

In [179]:
# Remove features with balance < 3%
remove = balance[balance['balance'] < 3].index.to_list()
droped = pd.concat([data[remove], droped], axis=1)
data = data.drop(remove, axis=1)


__2. Numerical Features__

In [180]:
numerical_features = [col for col in data.columns if col.startswith('n_')]

In [181]:
data[numerical_features].describe().T

,count,mean,std,min,25%,50%,75%,max
n_age,14942.0,26.015794,14.156582,0.00,19.00,23.00,31.000,87.00
n_income,14447.0,2230.816086,1566.527734,0.00,1470.00,1990.00,2790.000,10890.00
n_daysWithoutFrequency,14942.0,81.224936,144.199576,0.00,13.00,41.00,83.750,1745.00
n_lifetimeValue,14942.0,302.561871,364.319566,0.00,83.60,166.20,355.075,6727.80
n_frequencies,14916.0,40.120542,65.466459,1.00,7.00,18.00,45.000,1031.00
n_attendedClasses,14942.0,10.152456,29.154202,0.00,0.00,0.00,3.000,581.00
n_allowedWeeklyVisits,14407.0,5.759561,2.118867,1.00,4.00,7.00,7.000,7.00
n_allowedVisits,14942.0,41.636299,21.066166,0.56,25.72,38.99,60.970,240.03
n_realVisits,14942.0,5.320707,6.332958,0.00,1.00,4.00,7.000,84.00
n_renewals,14942.0,1.205260,1.381305,0.00,0.00,1.00,2.000,6.00


### __Dealing with missing values:__ 
- Median imputation for binary features
- KNN imputation for numerical features

In [182]:
def get_missing(data):
    return 100*data.isna().sum()[data.isna().sum() != 0].sort_values(ascending=False)/data.shape[0]

get_missing(data)

n_allowedWeeklyVisits    3.580511
n_income                 3.312810
b_waterActivities        0.247624
b_fitnessActivities      0.234239
b_teamActivities         0.234239
b_combatActivities       0.220854
n_frequencies            0.174006
dtype: float64

__1. Binary features:__ we will fill with the most frequent value (mode), because it will not skew the data too much and we have very few missing values. 

In [183]:
to_impute = ['b_waterActivities', 'b_fitnessActivities', 'b_teamActivities', 'b_combatActivities']
modes = data[to_impute].mode().iloc[0].to_dict()

# Impute missing values
data[to_impute] = data[to_impute].fillna(modes)

__2. Numerical features:__ lets use a KNN imputer, because it will be able to fill the missing values with a more accurate value than the mean or median.

In [184]:
# Lets start with getting the Nan indices
nan_indices = data[data['n_frequencies'].isna() | data['n_income'].isna() | data['n_allowedWeeklyVisits'].isna()].index

# First scale the numerical features
scaler = StandardScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# Now fit the KNNImputer to the numerical features
imputer = KNNImputer(n_neighbors=5)
data[numerical_features] = imputer.fit_transform(data[numerical_features])

# Now we can inverse the scaling
data[numerical_features] = scaler.inverse_transform(data[numerical_features])

In [185]:
data.loc[nan_indices, numerical_features]

,n_age,n_income,n_daysWithoutFrequency,n_lifetimeValue,n_frequencies,n_attendedClasses,n_allowedWeeklyVisits,n_allowedVisits,n_realVisits,n_renewals
0,60.0,5500.0,1.0,89.35,9.0,7.0,3.0,6.28,2.0,0.0
19,28.0,2510.0,10.0,52.00,2.0,0.0,7.0,24.01,2.0,0.0
34,74.0,5210.0,20.0,38.50,4.0,0.0,7.0,30.03,4.0,0.0
68,43.0,4036.0,1.0,849.80,214.0,0.0,7.0,21.98,9.0,3.0
76,9.0,0.0,5.0,708.20,64.0,64.0,2.0,17.42,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...
14874,15.0,0.0,62.0,353.60,20.0,17.0,1.0,8.71,0.0,0.0
14898,23.0,1980.0,0.0,68.20,28.0,0.0,7.0,31.99,19.0,0.0
14904,21.0,1820.0,3.0,54.80,9.0,0.0,7.0,21.98,5.0,0.0
14930,4.0,686.0,38.0,77.60,5.0,5.0,2.0,17.42,7.0,1.0


### __Feature engineering__

In [218]:
data.columns

Index(['n_age', 'b_gender', 'n_income', 'd_enrollmentStart',
       'd_enrollmentFinish', 'd_lastPeriodStart', 'd_lastPeriodFinish',
       'd_lastVisit', 'n_daysWithoutFrequency', 'n_lifetimeValue',
       'b_useByTime', 'b_waterActivities', 'b_fitnessActivities',
       'b_teamActivities', 'b_combatActivities', 'n_frequencies',
       'n_attendedClasses', 'n_allowedWeeklyVisits', 'n_allowedVisits',
       'n_realVisits', 'n_renewals', 'b_dropout', 'c_age'],
      dtype='object')

__Age class:__

In [286]:
data['c_age'] = pd.cut(data['n_age'], bins=[-1, 5, 20, 35, 49, 65, np.inf], labels=['<5','5-20', '20-35', '35-49', '49-65', '65+'])
data['c_age'].value_counts(dropna=False)

20-35    7246
5-20     3875
35-49    1750
<5        927
49-65     836
65+       308
Name: c_age, dtype: int64

It is quite odd that we have values for age that are bellow 5, but these are probably babies that are learning how to swim

__Enrollment duration + categorical:__

In [280]:
data['n_enrollmentDuration'] = (pd.to_datetime(data['d_enrollmentFinish']) - pd.to_datetime(data['d_enrollmentStart'])).dt.days
data['c_membershipDuration'] = pd.cut(data['n_enrollmentDuration'], bins=[-1, 30, 90, 180, 365, np.inf], labels=['<1m', '1-3m', '3-6m', '6-12m', '12m+'])
data['c_membershipDuration'].value_counts()

6-12m    3707
12m+     3657
<1m      2850
3-6m     2402
1-3m     2326
Name: c_membershipDuration, dtype: int64

We dont want enrollment durations of 0, as these are probably errors. We have to investigate this further.

In [318]:
data[data['n_enrollmentDuration'] == 0]['b_dropout'].mean()

nan

This means that if we have a customer that has an enrollment duration of 0 he never dropped out. Lets then impute the 0 values with the maximum value of the enrollment duration.

In [317]:
data['n_enrollmentDuration'] = data.apply(lambda x: (pd.to_datetime('2019-10-31') - pd.to_datetime(x['d_enrollmentStart'])).days if x['n_enrollmentDuration'] == 0 else x['n_enrollmentDuration'], axis=1)

__Last period duration:__

In [220]:
data['n_lastPeriodDuration'] = (pd.to_datetime(data['d_lastPeriodFinish']) - pd.to_datetime(data['d_lastPeriodStart'])).dt.days

__Days since last visit + categorical:__

In [321]:
print('Last data available on:', data['d_lastVisit'].value_counts().sort_index(ascending=False).index[0])
data['n_sinceLastVisit'] = (pd.to_datetime(data['d_lastVisit']).max() - pd.to_datetime(data['d_lastVisit'])).dt.days

data['c_sinceLastVisit'] = pd.cut(data['n_sinceLastVisit'], bins=[-1, 7, 14, 30, 60, 90, 365, np.inf], labels=['<1w', '1-2w', '2-4w', '1-2m', '2-3m', '3-12m', '>12m'])
data['c_sinceLastVisit'].value_counts(dropna=False)

Last data available on: 2019-10-31


>12m     9536
3-12m    2558
<1w      1929
1-2w      395
2-4w      262
1-2m      260
2-3m        2
Name: c_sinceLastVisit, dtype: int64

__Number of frequencies per week:__ 
- The number of frequencies is the number of times a customer went to the facility during its enrollment period.

In [326]:
data['n_frequenciesWeek'] = 7 * data['n_frequencies'] / (data['n_enrollmentDuration'])

In [330]:
data['n_allowedWeeklyVisits'].value_counts()

7.0    10779
2.0     2386
4.0      751
1.0      526
3.0      170
6.0      141
5.0       30
1.8       20
2.4       19
6.4       19
5.4       13
2.2       13
3.2       11
2.6       10
6.2        8
1.6        7
4.4        5
4.6        4
1.2        4
2.8        4
3.6        3
4.8        3
3.4        3
5.8        3
3.8        2
1.4        2
5.2        2
6.8        1
4.2        1
6.6        1
5.6        1
Name: n_allowedWeeklyVisits, dtype: int64